
# <span style="color:#006E7F">__Introduction to Oxford Nanopore Data Analysis__ <a class="anchor"></span>  


Created by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-INRAE) - Septembre 2021 Formation SouthGreen

Adapted by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) - Novembre 2022


# <span style="color:#006E7F">__TP3 - CONTIGS QUALITY__ <a class="anchor" id="data"></span>  
    
# <span style="color: #4CACBC;"> 1. Perform a comparison of assemblies using QUAST</span>  


The metrics of assemblies can be evaluated using a quality assessment tool such as [QUAST](http://quast.bioinf.spbau.ru/manual.html).

In part 2, we have assembled raw data using various assemblers and also polished and corrected it. 

Aggregate all assemblies you have produced for your favorite clone in a new folder named "AGGREGATED".

We're going to compare it with QUAST.

In [ ]:
cd ~/work/RESULTS
mkdir -p ~/work/RESULTS/AGGREGATED
cd ~/work/RESULTS/AGGREGATED
ln -s ~/work/RESULTS/FLYE/assembly.fasta Hh_FLYE.fasta
ln -s ~/work/RESULTS/FLYE_RACON/assembly.racon2.fasta Hh_FLYE_RACONx2.fasta
ls -l ~/work/RESULTS/AGGREGATED

### <span style="color: #4CACBD;"> 1.1  Run QUAST on CLONE assemblies and compare them </span>

In [ ]:
cd ~/work/RESULTS/AGGREGATED/
quast.py *.fasta -o QUAST

#### Go to the QUAST directory and check the file content.

#### Looking at the output statistics, what are the main differences between the assemblies?

#### Look total sizes, number of contigs, N50 statistics.

#### For your favorite clone what is the best one ?

## <span style="color: #4CACBD;"> 1.2  Comparison of the assemblies against the reference genome of closely organism (SEVERAL ASSEMBLIES!) </span>

It's possible to compare assemblies against the reference genome of closely related organism. 

In our case we will use the real dataset Hh. 

Previously calculated assemblies for Hh can be found in DATA/real_Hh/Hh-ASSEMBLIES (used CulebrONT)

In [ ]:
ls -lh ~/work/DATA/real_Hh/Hh-ASSEMBLIES

### Compare Hh assemblies against the reference sequence using QUAST

#### WARNING : This can take a while!! 

In [ ]:
mkdir ~/work/RESULTS/AGGREGATED_Hh/
cd ~/work/RESULTS/AGGREGATED_Hh/
time quast.py /home/jovyan/work/DATA/real_Hh/Hh-ASSEMBLIES/*MEDAKA_STARTFIXED-CIRCULARISED.fasta -R /home/jovyan/work/DATA/real_Hh/REFH_M1C132.fasta -o QUAST_REF

#### Which of the assemblies are closest in size relative to the reference ?

#### Does this have the largest N50 and fewest number of contigs?

#### What assembly does obtain the best contiguity metrics?


## <span style="color: #4CACBD;"> 2. Assessing gene space using BUSCO (REAL DATA) </span>


Benchmarking Universal Single-Copy Orthologs ([BUSCO](https://busco.ezlab.org/busco_userguide.html)) helps to check if you have a good assembly, by searching the expected single-copy lineage-conserved orthologs in any newly-sequenced genome from an appropriate phylogenetic clade.

Calculate the gene space for each assembly

In [ ]:
mkdir -p ~/work/RESULTS/AGGREGATED_Hh/BUSCO
cd ~/work/RESULTS/AGGREGATED_Hh/BUSCO

Lineage can be choose in busco database using --list-datasets parameter

In [ ]:
# busco env
conda activate busco

In [ ]:
busco --list-datasets

In [ ]:
ASSEMBLY="~/work/DATA/real_Hh/Hh-ASSEMBLIES/FLYE-STEP_CORRECTION_MEDAKA_STARTFIXED-CIRCULARISED.fasta"
LINEAGE=bacteria_odb10
busco -i "$ASSEMBLY" -l "$LINEAGE" -c "${CPUS:-8}" -m genome -o BUSCO_RESULTS 
# deactivate busco env
conda deactivate

### What does the gene space look like for this assembly?